In [1]:
import os
import pandas as pd
import json

def load_json_files_and_merge(base_directory, max_files_per_category=10):
    all_data = []

    for category in os.listdir(base_directory):
        category_path = os.path.join(base_directory, category)
        
        if os.path.isdir(category_path):
            print(f"카테고리 처리 중: {category}")
            file_count = 0
            for json_file in os.listdir(category_path):
                if json_file.endswith('.json') and file_count < max_files_per_category:
                    file_path = os.path.join(category_path, json_file)
                    
                    with open(file_path, 'r', encoding='utf-8') as file:
                        data = json.load(file)
                        
                        if 'SJML' in data and 'text' in data['SJML']:
                            for text_item in data['SJML']['text']:
                                text_item['category'] = category
                                all_data.append(text_item)
                    
                    file_count += 1
                
                if file_count >= max_files_per_category:
                    break

    df = pd.DataFrame(all_data)
    print("모든 데이터로 DataFrame 생성 완료")
    
    return df

base_directory = 'training_raw_data'

df = load_json_files_and_merge(base_directory)

print(df.shape)
df.head()

카테고리 처리 중: 사건사고
카테고리 처리 중: 경제
카테고리 처리 중: 국제
카테고리 처리 중: 스포츠
카테고리 처리 중: 연예
카테고리 처리 중: 건강
카테고리 처리 중: 사회일반
카테고리 처리 중: 지역
카테고리 처리 중: 여행레저
카테고리 처리 중: 산업
카테고리 처리 중: 여성복지
카테고리 처리 중: IT_과학
카테고리 처리 중: 취미
카테고리 처리 중: 교육
카테고리 처리 중: 문화
카테고리 처리 중: 정치
카테고리 처리 중: 라이프스타일
모든 데이터로 DataFrame 생성 완료
(16163, 9)


,title,subtitle,content,board,writer,write_date,url,source_site,category
0,강원 인제 카드형 지역상품권 출시…최대 10% 환급,,(인제=뉴스1) (이름) 기자 = 강원 인제지역에서 현금처럼 사용할 있는 인제사랑상...,지방_강원,(이름),2020-07-02 16:00:49,https://www.news1.kr/articles/?3984227,뉴스1,사건사고
1,영서 최고 36도 '찜통'…동해안도 무더위에 시민들 '헉헉',,"(강원=뉴스1) (이름) 기자,(이름) 기자,(이름) 기자 = 22일 강원 영서를 ...",지방_강원,(이름),2020-06-22 14:03:47,https://www.news1.kr/articles/?3972604,뉴스1,사건사고
2,춘천 육림연탄공장 '35년 만에 역사 속으로',,"(춘천=뉴스1) (이름) 기자 = 강원 영서북부권인 춘천, 홍천, 화천, 양구, 인...",지방_강원,(이름),2020-07-19 12:06:53,https://www.news1.kr/articles/?3999996,뉴스1,사건사고
3,경찰 n번방 구매자 첫 신상공개 결정…법원 판단 남겨,,(춘천=뉴스1) (이름) 기자 = 경찰이 성 착취 동영상 구매자의 신상 정보를 처음...,지방_강원,(이름),2020-07-02 21:01:32,https://www.news1.kr/articles/?3984580,뉴스1,사건사고
4,‘미래 20년을 준비’…건강보험심사평가원 제44회 심평포럼,,(원주=뉴스1) (이름) 기자 = 건강보험심사평가원(원장 (이름))은 2일 강원 원...,지방_강원,(이름),2020-07-02 18:18:55,https://www.news1.kr/articles/?3984477,뉴스1,사건사고


In [2]:
df.iloc[313]

title             국가상대 소송 '당사자, 패소해도 공익적이면 소송비용 깎아줘라'…법무검찰개혁위 권고
subtitle                                                        
content        국가 또는 행정청을 상대로하는 공익소송에서 국가가 패소당사자에게 소송비용을 회수하는...
board                                                      사회_법조
writer                                                      (이름)
write_date                                   2020-02-10 17:27:27
url            https://news.mt.co.kr/mtview.php?no=2020021016...
source_site                                              머니투데이뉴스
category                                                    사건사고
Name: 313, dtype: object

In [3]:
# 각 카테고리별 데이터 개수 계산
category_counts = df['category'].value_counts()

# 결과 출력
print("각 카테고리별 데이터 개수:")
print(category_counts)

각 카테고리별 데이터 개수:
category
문화        991
산업        985
지역        984
IT_과학     982
스포츠       981
여성복지      980
사건사고      977
교육        976
연예        973
국제        973
여행레저      953
라이프스타일    952
취미        948
경제        923
정치        921
사회일반      863
건강        801
Name: count, dtype: int64


In [4]:
def preprocess_dataframe(df, sample_size=100, random_state=42):
    # 불필요한 컬럼 제거 및 컬럼명 변경
    filtered_df = df.drop(columns=['subtitle', 'board', 'writer'])
    filtered_df = filtered_df.rename(columns={'source_site': 'writer'})
    
    # 각 카테고리별로 균등하게 샘플 수 계산
    category_counts = filtered_df['category'].value_counts()
    num_categories = len(category_counts)
    base_samples_per_category = sample_size // num_categories
    
    # 남은 샘플 수 계산
    remaining_samples = sample_size % num_categories
    
    # 각 카테고리별 샘플 수 리스트 생성
    samples_per_category = [base_samples_per_category] * num_categories
    
    # 남은 샘플을 카테고리에 균등하게 분배
    for i in range(remaining_samples):
        samples_per_category[i] += 1
    
    # 각 카테고리별로 샘플링
    sampled_dfs = []
    for category, count in zip(category_counts.index, samples_per_category):
        category_df = filtered_df[filtered_df['category'] == category]
        sampled_category = category_df.sample(n=min(count, len(category_df)), random_state=random_state)
        sampled_dfs.append(sampled_category)
    
    # 최종 샘플링된 데이터프레임을 섞음
    sampled_df = pd.concat(sampled_dfs).sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    print(f"{sample_size}개의 행이 각 카테고리에서 골고루 추출되었습니다. 현재 DataFrame의 크기: {sampled_df.shape}")
    print("각 카테고리별 샘플 수:")
    print(sampled_df['category'].value_counts())
    
    return sampled_df

def save_to_csv(df, output_path='data/1_filtered_data.csv'):
    df.to_csv(output_path)
    print(f"전처리된 데이터가 {output_path}에 저장되었습니다.")

# 데이터 전처리 및 샘플링
sampled_df = preprocess_dataframe(df, sample_size=1000)

# CSV 파일로 저장
save_to_csv(sampled_df)

# 변경된 DataFrame 출력
sampled_df

1000개의 행이 각 카테고리에서 골고루 추출되었습니다. 현재 DataFrame의 크기: (1000, 6)
각 카테고리별 샘플 수:
category
연예        59
취미        59
라이프스타일    59
사건사고      59
여행레저      59
지역        59
경제        59
산업        59
여성복지      59
교육        59
스포츠       59
IT_과학     59
국제        59
문화        59
정치        58
건강        58
사회일반      58
Name: count, dtype: int64
전처리된 데이터가 data/1_filtered_data.csv에 저장되었습니다.


,title,content,write_date,url,writer,category
0,"업텐션 '6주년, 허니텐 있기에 가능..'spin off' 청량+섹시 공존'",. 아이돌그룹 업텐션이 컴백 신곡 ';spin off';로 청량함과 섹시함이 공존하...,2021-06-14 15:17:56,https://star.mt.co.kr/stview.php?no=2021061412...,스타뉴스,연예
1,일본 '포스트 아베' 유력주자 '미군 전술핵 배치 논의해야' 파장,【서울=뉴시스】(이름) 기자 = 일본 '포스트 아베'의 유력주자 중 하나인 이시바 ...,2017-09-06 15:44:38,https://newsis.com/view/?id=NISX20170906_00000...,뉴시스,취미
2,"文대통령, 정무 (이름)·국정홍보 여현호 등 靑 비서관 6명 임명",【서울=뉴시스】(이름) 기자 = (이름) 대통령은 9일 정무비서관에 (이름) 전 아...,2019-01-09 18:00:21,https://newsis.com/view/?id=NISX20190109_00005...,뉴시스,취미
3,"거래소, 현대상선 불성실공시법인 지정",【서울=뉴시스】(이름) 기자 = 한국거래소 유가증권시장본부는 29일 배임 혐의 발생...,2018-01-29 19:46:36,https://newsis.com/view/?id=NISX20180129_00002...,뉴시스,라이프스타일
4,"[오늘의 주요 일정] 강원(11일, 목)",(강원=뉴스1) = ◇강원도-(이름) 도지사11:00 소부장기업 기술자립 지원 프로...,2020-06-11 05:00:00,https://www.news1.kr/articles/?3961722,뉴스1,사건사고
...,...,...,...,...,...,...
995,워너원과 함께하는 '2017 그린 크리스마스 캠페인',. . . . . . 이니스프리는 '2017 그린 크리스마스' 캠페인을 진행하며 워...,2017-11-24 17:30:00,https://moneys.mt.co.kr/news/mwView.php?no=201...,머니S,산업
996,"도쿄올림픽 참가 선수, 최소 나흘에 1차례 코로나19 검사한다",(서울=뉴스1) (이름) 기자 = 도쿄올림픽에 출전하는 선수들이 최소 나흘에 1번씩...,2021-03-11 14:43:07,https://www.news1.kr/articles/?4237885,뉴스1,스포츠
997,"국회 본회의, 한국당 '보이콧'에 30분간 정회(상보)",국회가 내년도 예산안과 법인세·소득세법 개정안 의결을 위한 본회의를 속개한지 30여...,2017-12-05 22:46:52,https://news.mt.co.kr/mtview.php?no=2017120522...,머니투데이뉴스,정치
998,잇단 산재사망 현대중공업 특별근로감독 받는다,(세종=뉴스1) (이름) 기자 = 최근 수년간 산업재해 사망자가 잇따라 발생한 현대...,2016-10-19 22:14:41,https://www.news1.kr/articles/?2807172,뉴스1,교육


In [5]:
# 고유한 카테고리 나열
unique_categories = sampled_df['category'].unique()
print("고유한 카테고리 목록:")
for category in unique_categories:
    print(f"- {category}")

고유한 카테고리 목록:
- 연예
- 취미
- 라이프스타일
- 사건사고
- 여행레저
- 정치
- 지역
- 경제
- 산업
- 건강
- 사회일반
- 스포츠
- 여성복지
- 국제
- IT_과학
- 교육
- 문화
